In [1]:
#!pip install --upgrade transformers

#### HuggingFace token
This token is mandatory. Register on the website and generate one.

In [2]:
access_token = "secret"

In [3]:
from huggingface_hub import login
login(token=access_token)

/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### 1. Importing libraries

In [4]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel, TFBertForQuestionAnswering,TFAutoModelWithLMHead

In [5]:
GPU_use = 2
st = "cuda:"+str(GPU_use)
print(torch.cuda.current_device())
torch.cuda.set_device(GPU_use) 
print(torch.cuda.current_device())


#st = "cpu"
#torch.device(st)

0
2


### 2. Choose the model

In [6]:
base_model = 'meta-llama/Meta-Llama-3-8B'
#base_model = 'meta-llama/Llama-3.2-1B'

### 3.Importing the tokenizer

In [7]:
tokr = AutoTokenizer.from_pretrained(base_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
toks = tokr("ciao come stai?",return_tensors="pt")

In [9]:
toks

{'input_ids': tensor([[128000,     66,  23332,   2586,    357,   2192,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [10]:
tokr.batch_decode(toks['input_ids'])

['<|begin_of_text|>ciao come stai?']

### 4. importing the model

In [11]:
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.bfloat16, device_map=GPU_use,
                                             token=access_token)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.33it/s]


### 5. setting the correct prompt (different for each models)

In [12]:
fmt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: {}
===
{}
ASSISTANT:"""

def sql_prompt(d): return fmt.format(d["context"], d["question"])
def question(table, quest):
    tst = {'context' : '', 'question':''}
    tst['context'] = table
    tst['question'] = quest
    return sql_prompt(tst)

### 6. play with the model 

In [13]:
%%time

t = 'Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry\'s standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.'
q = 'translate from english to italian'


#t =''
#q = 'hi, how are you?'
#print(question(t,q))
test = question(t,q)
toks = tokr(test, return_tensors="pt")
res = model.generate(**toks.to(st), max_new_tokens=100, top_p = 0).to('cpu')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CPU times: user 2.8 s, sys: 342 ms, total: 3.14 s
Wall time: 3.14 s


In [14]:
#%%time
print(tokr.batch_decode(res)[0].replace("*","\n"))

<|begin_of_text|>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.
===
translate from english to italian
ASSISTANT: Lorem Ipsum è semplicemente un testo di esempio utilizzato nel settore della stampa e della composizione tipografica. Lorem Ipsum è stato il testo di esempio standard della stampa sin dal 1500, 